In [20]:
!pip install -U StanfordCoreNLP
!pip install -U pycorenlp

Requirement already up-to-date: StanfordCoreNLP in /Users/poornimajoshi/anaconda3/lib/python3.6/site-packages (3.9.1.1)
Requirement not upgraded as not directly required: requests in /Users/poornimajoshi/anaconda3/lib/python3.6/site-packages (from StanfordCoreNLP) (2.18.4)
Requirement not upgraded as not directly required: psutil in /Users/poornimajoshi/anaconda3/lib/python3.6/site-packages (from StanfordCoreNLP) (5.4.5)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /Users/poornimajoshi/anaconda3/lib/python3.6/site-packages (from requests->StanfordCoreNLP) (3.0.4)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /Users/poornimajoshi/anaconda3/lib/python3.6/site-packages (from requests->StanfordCoreNLP) (2.6)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /Users/poornimajoshi/anaconda3/lib/python3.6/site-packages (from requests->StanfordCoreNLP) (1.22)
Requirement not upgraded as not directly required: certif

In [1]:
# Read in data file
import pandas as pd
data = pd.read_csv('data.tsv', sep='\t')
data.set_index(data.columns[0], inplace=True)

data.head(5)

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL
ID,,,,,,,,
development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...
development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie
development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band)
development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera


In [16]:
data['Text'][2]

"He had been reelected to Congress, but resigned in 1990 to accept a post as Ambassador to Brazil. De la Sota again ran for governor of C*rdoba in 1991. Defeated by Governor Angeloz by over 15%, this latter setback was significant because it cost De la Sota much of his support within the Justicialist Party (which was flush with victory in the 1991 mid-terms), leading to President Carlos Menem 's endorsement of a separate party list in C*rdoba for the 1993 mid-term elections, and to De la Sota's failure to regain a seat in Congress."

In [2]:
len(data)

2000

In [3]:
data.isna().sum()

Text              0
Pronoun           0
Pronoun-offset    0
A                 0
A-offset          0
B                 0
B-offset          0
URL               0
dtype: int64

In [53]:
from pycorenlp import StanfordCoreNLP
import numpy as np

nlp = StanfordCoreNLP('http://localhost:9000')

resolved_lst = []
def resolve(corenlp_output, pronoun, index, develop_num):
    #print(corenlp_output)
    """ Transfer the word form of the antecedent to its associated pronominal anaphor(s) """
    for coref in corenlp_output['corefs']:
        mentions = corenlp_output['corefs'][coref]
        antecedent = mentions[0]  # the antecedent is the first mention in the coreference chain
        #print(antecedent)
        for j in range(1, len(mentions)):
            mention = mentions[j]
            target_sentence_original = mention['sentNum']
            target_token_original = mention['startIndex'] - 1
            offset_original_text = corenlp_output['sentences'][target_sentence_original - 1]['tokens'][target_token_original]
            
            if mention['text'] == pronoun and offset_original_text['characterOffsetBegin'] == index:
                #print("I am here for mention:",mention)
                #print("I am the pronoun", pronoun)
                
                # get the attributes of the target mention in the corresponding sentence
                #target_sentence = mention['sentNum']
                #target_token = mention['startIndex'] - 1
                
                antecedent_sentence = antecedent['sentNum']
                antecedent_token = antecedent['startIndex'] - 1
                # transfer the antecedent's word form to the appropriate token in the sentence
                #print("Here to see word:",corenlp_output['sentences'][antecedent_sentence - 1]['tokens'][antecedent_token]['characterOffsetBegin'])
                #corenlp_output['sentences'][target_sentence - 1]['tokens'][target_token]['word'] = antecedent['text']
                
                #print("Checking :", corenlp_output['sentences'][target_sentence - 1]['tokens'][target_token])

                resolved_offset = corenlp_output['sentences'][antecedent_sentence - 1]['tokens'][antecedent_token]['characterOffsetBegin']
                #print("antecedent", antecedent)
                #cols = ['ID', 'A', 'B', 'NEITHER']
                #d = pd.DataFrame(0, index=np.arange(len(data)), columns=cols)
                #df.to_csv('out.csv')
                resolved_lst.append((develop_num,resolved_offset))
                return resolved_lst


def print_resolved(corenlp_output):
    """ Print the "resolved" output """
    possessives = ['her', 'His', 'his', 'She', 'she', 'him', 'He', 'Her', 'he']
    for sentence in corenlp_output['sentences']:
        for token in sentence['tokens']:
            output_word = token['word']
            # check lemmas as well as tags for possessive pronouns in case of tagging errors
            if token['lemma'] in possessives or token['pos'] == 'PRP$':
                output_word += "'s"  # add the possessive morpheme
            output_word += token['after']
            print(output_word, end='')





In [54]:
for row in data.iterrows():
    #resolved_text = resolved_text.append([row[1]['Text']])
    output = nlp.annotate((row[1]['Text']), properties= {'annotators':'dcoref','outputFormat':'json','ner.useSUTime':'false'})
    #print(row[1][1], row[1][2])
    print(row[0])
    resolved_list = resolve(output, row[1][1], row[1][2], row[0])
    #print('Original:\n')
    #print((row[1]['Text']))
    #print('Resolved: \n')
    #print_resolved(output)
    

development-1
development-2
development-3
development-4
development-5
development-6
development-7
development-8
development-9
development-10
development-11
development-12
development-13
development-14
development-15
development-16
development-17
development-18
development-19
development-20
development-21
development-22
development-23
development-24
development-25
development-26
development-27
development-28
development-29
development-30
development-31
development-32
development-33
development-34
development-35
development-36
development-37
development-38
development-39
development-40
development-41
development-42
development-43
development-44
development-45
development-46
development-47
development-48
development-49
development-50
development-51
development-52
development-53
development-54
development-55
development-56
development-57
development-58
development-59
development-60
development-61
development-62
development-63
development-64
development-65
development-66
development-67
deve

development-520
development-521
development-522
development-523
development-524
development-525
development-526
development-527
development-528
development-529
development-530
development-531
development-532
development-533
development-534
development-535
development-536
development-537
development-538
development-539
development-540
development-541
development-542
development-543
development-544
development-545
development-546
development-547
development-548
development-549
development-550
development-551
development-552
development-553
development-554
development-555
development-556
development-557
development-558
development-559
development-560
development-561
development-562
development-563
development-564
development-565
development-566
development-567
development-568
development-569
development-570
development-571
development-572
development-573
development-574
development-575
development-576
development-577
development-578
development-579
development-580
development-581
developm

development-1031
development-1032
development-1033
development-1034
development-1035
development-1036
development-1037
development-1038
development-1039
development-1040
development-1041
development-1042
development-1043
development-1044
development-1045
development-1046
development-1047
development-1048
development-1049
development-1050
development-1051
development-1052
development-1053
development-1054
development-1055
development-1056
development-1057
development-1058
development-1059
development-1060
development-1061
development-1062
development-1063
development-1064
development-1065
development-1066
development-1067
development-1068
development-1069
development-1070
development-1071
development-1072
development-1073
development-1074
development-1075
development-1076
development-1077
development-1078
development-1079
development-1080
development-1081
development-1082
development-1083
development-1084
development-1085
development-1086
development-1087
development-1088
development-10

development-1513
development-1514
development-1515
development-1516
development-1517
development-1518
development-1519
development-1520
development-1521
development-1522
development-1523
development-1524
development-1525
development-1526
development-1527
development-1528
development-1529
development-1530
development-1531
development-1532
development-1533
development-1534
development-1535
development-1536
development-1537
development-1538
development-1539
development-1540
development-1541
development-1542
development-1543
development-1544
development-1545
development-1546
development-1547
development-1548
development-1549
development-1550
development-1551
development-1552
development-1553
development-1554
development-1555
development-1556
development-1557
development-1558
development-1559
development-1560
development-1561
development-1562
development-1563
development-1564
development-1565
development-1566
development-1567
development-1568
development-1569
development-1570
development-15

development-1995
development-1996
development-1997
development-1998
development-1999
development-2000


In [49]:
#lst = set(resolved_list)

In [55]:
len(resolved_list)

1942

In [56]:
resolved_list

[('development-1', 170),
 ('development-2', 228),
 ('development-3', 164),
 ('development-5', 219),
 ('development-6', 0),
 ('development-7', 152),
 ('development-8', 173),
 ('development-9', 128),
 ('development-10', 196),
 ('development-11', 212),
 ('development-12', 217),
 ('development-13', 273),
 ('development-14', 275),
 ('development-15', 219),
 ('development-16', 114),
 ('development-17', 121),
 ('development-18', 54),
 ('development-19', 97),
 ('development-20', 17),
 ('development-21', 113),
 ('development-22', 175),
 ('development-23', 91),
 ('development-24', 273),
 ('development-25', 266),
 ('development-26', 92),
 ('development-27', 238),
 ('development-28', 88),
 ('development-29', 177),
 ('development-30', 49),
 ('development-31', 0),
 ('development-32', 374),
 ('development-33', 283),
 ('development-34', 11),
 ('development-35', 206),
 ('development-36', 201),
 ('development-37', 215),
 ('development-38', 66),
 ('development-39', 24),
 ('development-40', 181),
 ('devel

In [58]:
df = pd.DataFrame(resolved_list, columns=['ID', 'OFFSET']) 


In [ ]:
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "tokenize,ssplit,pos,lemma,parse,sentiment" -port 9000 -timeout 300000000

In [63]:
res1 = pd.read_csv('resolved_words_with_offset.csv')
res1 = res1.drop(columns=['Unnamed: 0'])

res1.to_csv('resolved_words_with_offset.csv')
res1.head(50)

,ID,OFFSET
0,development-1,170
1,development-2,228
2,development-3,164
3,development-5,219
4,development-6,0
5,development-7,152
6,development-8,173
7,development-9,128
8,development-10,196
9,development-11,212
